<a href="https://colab.research.google.com/github/CooperJB710/SQL-Lite--SuperHero/blob/main/python/superhero_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Superhero Database Assignment (SQLite & Google Colab)

## Objective
In this assignment, you will design and implement a **Superhero Database** using SQLite in Google Colab. This database will store superheroes, villains, and their battles. You will write SQL queries to analyze battle outcomes and relationships between heroes and villains.

### Group Size:
- 2 to 3 students per group.

### Tools Required:
- **Google Colab**
- **SQLite (built into Python)**
- **CSV files (provided for import)**

## Step 1: Setting Up Your SQLite Database

### Tasks:
1. Initialize SQLite in Google Colab.
2. Create a database named `superhero_db.sqlite`.
3. Define the database schema, ensuring it includes the following tables:
   - `heroes`: Stores superhero names, powers, and teams.
   - `villains`: Stores villains and their evil plans.
   - `battles`: Tracks battles between heroes and villains, including the outcome.
4. Implement primary keys and foreign keys where appropriate.

✅ **Checkpoint:** Ensure that the database and tables have been successfully created.

In [ ]:
import sqlite3

# Create a connection to (or create) the SQLite database file
conn = sqlite3.connect('superhero_db.sqlite')
cursor = conn.cursor()

print("Database connection established!")

## Step 2: Import Data from CSV Files

### Tasks:
1. Download the provided CSV files.
2. Upload them to Google Colab.
3. Write a script to import data from these files into the appropriate tables.
4. Verify that the data has been successfully inserted.

✅ **Checkpoint:** Ensure that each table contains the expected data from the CSV files.

In [ ]:
import pandas as pd

# 1. Drop tables if they already exist (prevents unique constraint or leftover data on repeated runs)
cursor.execute("DROP TABLE IF EXISTS battles;")
cursor.execute("DROP TABLE IF EXISTS heroes;")
cursor.execute("DROP TABLE IF EXISTS villains;")
conn.commit()

# 2. Create the heroes, villains, and battles tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS heroes (
    hero_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    power TEXT,
    team TEXT
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS villains (
    villain_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    evil_plan TEXT
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS battles (
    battle_id INTEGER PRIMARY KEY AUTOINCREMENT,
    hero_id INTEGER NOT NULL,
    villain_id INTEGER NOT NULL,
    outcome TEXT CHECK(outcome IN ('hero_won','villain_won','draw')),
    FOREIGN KEY (hero_id) REFERENCES heroes(hero_id),
    FOREIGN KEY (villain_id) REFERENCES villains(villain_id)
);
""")

conn.commit()

# 3. Read & import data from CSV files

# --- HEROES ---
heroes_df = pd.read_csv('heroes.csv')

# If your CSV has an 'id' column but your table expects 'hero_id', rename it
if 'id' in heroes_df.columns:
    heroes_df.rename(columns={'id': 'hero_id'}, inplace=True)

# Insert heroes data
heroes_df.to_sql('heroes', conn, if_exists='append', index=False)

# --- VILLAINS ---
villains_df = pd.read_csv('villains.csv')

# If your CSV has 'id', rename it to 'villain_id' if needed
if 'id' in villains_df.columns:
    villains_df.rename(columns={'id': 'villain_id'}, inplace=True)

# Insert villains data
villains_df.to_sql('villains', conn, if_exists='append', index=False)

# --- BATTLES ---
battles_df = pd.read_csv('battles.csv')

# If your CSV has an extra 'id' column or something that doesn't match the schema, drop it
if 'id' in battles_df.columns:
    battles_df.drop(columns=['id'], inplace=True)

# OPTIONAL: Strip spaces in 'outcome' to avoid hidden whitespace issues
battles_df['outcome'] = battles_df['outcome'].astype(str).str.strip()

# Filter out any invalid outcomes to avoid CHECK constraint errors
allowed_outcomes = ['hero_won', 'villain_won', 'draw']
battles_df = battles_df[battles_df['outcome'].isin(allowed_outcomes)]

# Insert battles data
battles_df.to_sql('battles', conn, if_exists='append', index=False)

print("Heroes, villains, and battles data loaded successfully!")

# (Optional) Keep the connection open for querying below.
# If you want to close it and re-open for queries, uncomment:
# conn.close()

## Step 3: Querying the Database

### Tasks:
Write SQL queries to answer the following:
1. Retrieve all superheroes and their powers.
2. Find all battles where the hero won.
3. List all villains and their evil plans.
4. Determine which superhero has fought the most battles.

✅ **Checkpoint:** Ensure that all queries return meaningful results from the database.

In [ ]:
# 1. Retrieve all superheroes and their powers
query_all_heroes = """
SELECT name, power
FROM heroes;
"""
cursor.execute(query_all_heroes)
results = cursor.fetchall()

print("All Heroes and Their Powers:")
if results:
    for row in results:
        print(row)
else:
    print("No heroes found.")

In [ ]:
# 2. Find all battles where the hero won
query_hero_wins = """
SELECT b.battle_id,
       h.name AS hero,
       v.name AS villain,
       b.outcome
FROM battles b
JOIN heroes h ON b.hero_id = h.hero_id
JOIN villains v ON b.villain_id = v.villain_id
WHERE b.outcome = 'hero_won';
"""
cursor.execute(query_hero_wins)
results = cursor.fetchall()

print("\nBattles Where the Hero Won:")
if results:
    for row in results:
        print(row)
else:
    print("No battles found where hero won.")

In [ ]:
# 3. List all villains and their evil plans
query_villains = """
SELECT name, evil_plan
FROM villains;
"""
cursor.execute(query_villains)
results = cursor.fetchall()

print("\nVillains and Their Evil Plans:")
if results:
    for row in results:
        print(row)
else:
    print("No villains found.")

In [ ]:
# 4. Determine which superhero has fought the most battles
query_most_battles = """
SELECT h.name,
       COUNT(*) AS total_battles
FROM battles b
JOIN heroes h ON b.hero_id = h.hero_id
GROUP BY h.name
ORDER BY total_battles DESC
LIMIT 1;
"""
cursor.execute(query_most_battles)
result = cursor.fetchone()

print("\nSuperhero with the Most Battles:")
if result is not None:
    print(f"{result[0]} with {result[1]} battles.")
else:
    print("No battles found or no data in the table.")

## Step 4: Add a Custom Feature

### Tasks:
1. Design and implement an additional table related to superheroes or villains.
2. Populate it with data.
3. Write at least one query to retrieve meaningful insights from the new table.

**Examples of Custom Features:**
- Sidekicks for superheroes
- Villain hideouts
- Battle locations
- Superhero equipment

✅ **Checkpoint:** Ensure that the new table and queries function correctly.

In [ ]:
# Drop the table if it already exists (for testing purposes)
cursor.execute("DROP TABLE IF EXISTS sidekicks;")
conn.commit()

create_sidekicks_table = """
CREATE TABLE IF NOT EXISTS sidekicks (
    sidekick_id INTEGER PRIMARY KEY AUTOINCREMENT,
    sidekick_name TEXT NOT NULL,
    hero_id INTEGER,
    FOREIGN KEY (hero_id) REFERENCES heroes(hero_id)
);
"""

cursor.execute(create_sidekicks_table)
conn.commit()

print("Custom table 'sidekicks' created successfully!")

In [ ]:
# Insert some example sidekick data manually (or from another CSV)
sidekicks_data = [
    ("Robin", 2),   # Suppose hero_id=2 is Batman
    ("Falcon", 3),  # Suppose hero_id=3 is Iron Man (just for example)
    ("Kid Flash", 1) # Suppose hero_id=1 is Superman
]

insert_sidekick = """
INSERT INTO sidekicks (sidekick_name, hero_id)
VALUES (?, ?);
"""
cursor.executemany(insert_sidekick, sidekicks_data)
conn.commit()

print("Sidekicks inserted into table successfully!")

## Bonus Challenge (+5 Points)

### Tasks:
1. Enhance your project by adding a visual or interactive component.

**Options:**
- Visualize battles using Matplotlib (e.g., a bar chart showing how many battles each superhero has fought).
- Make it interactive (e.g., allow users to input a superhero name and display their battles).

✅ **Checkpoint:** If attempting the bonus, ensure that your visualization or interactive component works correctly.

## Submission Guidelines

Each group must submit a **Google Colab Notebook (.ipynb)** containing:
- ✅ **Database schema (DDL)**
- ✅ **CSV imports & data insertion (DML)**
- ✅ **SQL queries & results**
- ✅ **Custom feature (bonus if applicable)**

## Grading Criteria (Total: 25 Points)

| Section                 | Points |
|-------------------------|--------|
| Database Setup          | 5      |
| Data Importing         | 10     |
| Querying the Data      | 10     |
| Extra Feature          | 5      |
| Bonus Challenge (Optional) | +5  |

### Final Notes:
- Be **creative** when designing superheroes and villains.
- **Comment your code** to explain its functionality.
- **Have fun** building your superhero universe!